In [1]:
# !pip install torch transformers datasets evaluate accelerate timm ipykernel ipywidgets IProgress scikit-learn scipy peft bitsandbytes trl streamlit

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# this restricts libs to only only allow specified gpus to be used
import torch
import os
import numpy as np
import random
CUDA_DEVICE_IDX = 5
os.environ["CUDA_VISIBLE_DEVICES"] = f"{CUDA_DEVICE_IDX}"
device_map = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_map)

def set_seed(random_seed: int = 42):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

# Choosing model

In [4]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
checkpoint = "ai-forever/rugpt3large_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# Dataset preprocessing

In [5]:
# uncomment this to download Dostaevskiy's text corpus
# !wget https://gitlab.com/z00logist/artificial-dostoevsky/-/raw/main/data/corpus.txt

In [6]:
raw_text = open('corpus.txt').read()
print('Text example:', raw_text[:300])
print('Length of full corpus:', len(raw_text))

Text example: Ох уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь... невольно задумаешься, а там всякая дребедень и пойдет в голову; право бы, запретил им писать; так-таки п
Length of full corpus: 9652088


# Let's make Train/Validation/Test splits

In [7]:
from collections import OrderedDict
from datasets import Dataset, DatasetDict
# let's divide text into chunks of roughly 10.000 chars,
# then shuffle them between splits
document_size = 10000
print('Number of documents:', len(raw_text) / document_size)

def get_split_idx(sizes: list | tuple, size: int):
    """Returns idx for split according to distributions of sizes"""
    idx = torch.arange(0, size)
    probabilities = torch.tensor(sizes)
    probabilities /= probabilities.sum()
    split_mask = torch.multinomial(probabilities, num_samples=size, replacement=True)
    return tuple((idx[split_mask == i].numpy().tolist() for i in range(len(sizes))))

def get_dataset_dict_old(text: str, sizes_dict: OrderedDict):
    """Returns the splitted dataset according to sizes_dict (old code for reproducibility)"""
    split_idx = get_split_idx(list(sizes_dict.values()), int(len(text) / document_size))
    split_idx = {split: split_idx[i] for i, split in enumerate(sizes_dict)}
    datasets = DatasetDict() 
    for split, idx in split_idx.items():
        datasets[split] = Dataset.from_dict({'text': [text[i * document_size: i * document_size + document_size] for i in idx], 'start_idx': [i * document_size for i in idx]})
    return datasets

def get_dataset_dict_updated(text: str, train_size=0.8, seed=52):
    """Returns the splitted dataset according to train_size"""
    datasets = Dataset.from_dict({'text': [text[i: i + document_size] for i in range(0, len(text), document_size)], 
                                           'start_idx': [i for i in range(0, len(text), document_size)]})
    datasets = datasets.train_test_split(train_size=train_size, seed=seed)
    val_test_datasets = datasets.pop('test').train_test_split(train_size=0.5, seed=seed)
    datasets['validation'] = val_test_datasets.pop('train')
    datasets['test'] = val_test_datasets.pop('test')
    return datasets

set_seed(52)
datasets = get_dataset_dict_old(raw_text, OrderedDict([('train', 0.8), ('validation', 0.1), ('test', 0.1)]))
datasets

Number of documents: 965.2088


DatasetDict({
    train: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 757
    })
    validation: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 91
    })
    test: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 117
    })
})

In [8]:
# let's check (chunk_sizes are not all the same though)
print(datasets['train'][7]['text'])
print(f"start_idx: {datasets['train'][7:10]['start_idx']}, chunk_sizes: {list(map(len, datasets['train'][7:10]['text']))}")

а! Цена всех одиннадцати книг, присовокупив сюда издержки на переплет, была по крайней мере рублей шестьдесят. Где взять денег? Я думала-думала и не знала, на что решиться. У матушки просить не хотелось. Конечно, матушка мне непременно бы помогла; но тогда все бы в доме узнали о нашем подарке; да к тому же этот подарок обратился бы в благодарность, в плату за целый год трудов Покровского. Мне хотелось подарить одной, тихонько от всех. А за труды его со мною я хотела быть ему навсегда одолженною без какой бы то ни было уплаты, кроме дружбы моей. Наконец я выдумала, как выйти из затруднения. Я знала, что у букинистов в Гостином дворе можно купить книгу иногда в полцены дешевле, если только поторговаться, часто малоподержанную и почти совершенно новую. Я положила непременно отправиться в Гостиный двор. Так и случилось; назавтра же встретилась какая-то надобность и у нас и у Анны Федоровны. Матушке понездоровилось, Анна Федоровна очень кстати поленилась, так что пришлось все поручения возл

In [9]:
# tokenizing before chunking
def tokenize_function(examples):
    return tokenizer(examples['text'])
tokenized_datasets = datasets.map(tokenize_function, remove_columns=['start_idx', 'text'], batched=True).remove_columns('attention_mask')
tokenized_datasets

Map:   0%|          | 0/757 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2617 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 757
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 91
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 117
    })
})

In [10]:
# let's see what we get
print(f"document_sizes: {list(map(len, tokenized_datasets['train'][0:10]['input_ids']))}")

document_sizes: [2617, 2566, 2578, 2529, 2380, 2325, 2352, 2376, 2598, 2631]


In [11]:
# now let's take these documents and chunk them up each, chunks will be of size 1024. In order to fit in RAM restrictions, we'll use IterableDataset and use generator
from datasets import IterableDataset, IterableDatasetDict, Features, Value, Sequence
from random import randint
from functools import partial
chunk_size = 1024

def chunk_generator(dataset: Dataset, sequential: bool = False):
    """Generator that (maybe randomly) picks documents from the dataset and then random chunk of size chunk_size"""
    if sequential:
        for document_idx in range(len(dataset)):
            tokens = dataset[document_idx]['input_ids']
            i = randint(0, len(tokens) - chunk_size)
            yield {'input_ids': tokens[i: i + chunk_size]}
    else:
        while True:
            document_idx = randint(0, len(dataset) - 1)
            tokens = dataset[document_idx]['input_ids']
            i = randint(0, len(tokens) - chunk_size)
            yield {'input_ids': tokens[i: i + chunk_size]}

set_seed(2718)
# it's iterable, so it's light-weight
chunked_data = IterableDatasetDict()
for split in tokenized_datasets:
    dataset = tokenized_datasets[split]
    chunked_data[split] = (IterableDataset if split == 'train' else Dataset).from_generator(
        partial(chunk_generator, dataset=dataset, sequential=split != 'train'),
        features=Features({'input_ids': Sequence(feature=Value(dtype='int32'))})
    )
tokenized_datasets = chunked_data
chunked_data

Generating train split: 0 examples [00:00, ? examples/s]

IterableDatasetDict({
    train: IterableDataset({
        features: ['input_ids'],
        num_shards: 1
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 91
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 117
    })
})

In [12]:
chunked_data['train'].features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}

In [13]:
import psutil
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")


RAM used: 1252.09 MB


In [14]:
# let's check dtype of the dataset
type(list(tokenized_datasets['train'].take(1))[0]['input_ids'])

list

In [15]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # does job for creating attention_mask and labels for us (it just copies inputs for labels, bcause shifting happens inside the model)

In [16]:
out = data_collator(list(tokenized_datasets["train"].take(5)))
out

{'input_ids': tensor([[   16,  1012,   784,  ...,   669,  4223,   309],
        [13134,  2625,   519,  ...,  1441,    16,   687],
        [  299,  1186,  1248,  ...,   433,   968,    18],
        [   16,   670,    16,  ...,  1335, 35338,    18],
        [ 2754,  5804,   289,  ...,   503,  1042,    18]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[   16,  1012,   784,  ...,   669,  4223,   309],
        [13134,  2625,   519,  ...,  1441,    16,   687],
        [  299,  1186,  1248,  ...,   433,   968,    18],
        [   16,   670,    16,  ...,  1335, 35338,    18],
        [ 2754,  5804,   289,  ...,   503,  1042,    18]])}

# Training

In [17]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

## LoRA

In [25]:
from transformers import BitsAndBytesConfig
from trl import SFTTrainer

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map=device_map
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16, # 8-256
    lora_alpha=32, # (>= r) alpha/r ~ 2-4
    target_modules=["c_attn", "c_proj"], 
    lora_dropout=0.05, # 0.01-0.2
    # bias="none", # train only LoRA weights, no biases 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [26]:
model.print_trainable_parameters()

trainable params: 6,488,064 || all params: 766,788,096 || trainable%: 0.8461


In [ ]:
training_args = TrainingArguments(
    output_dir="./rugpt3large_lora_results_long",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    fp16=True,
    max_grad_norm=0.5,
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    max_steps=1200,
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=25,
    logging_dir="./rugpt3large_lora_logs",
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    peft_config=peft_config,
    processing_class=tokenizer
)

Truncating eval dataset:   0%|          | 0/91 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [29]:
set_seed(11)
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,3.242400,3.177843
100,3.205500,3.160163
150,3.185100,3.149015
200,3.170100,3.141162
250,3.141900,3.133629
300,3.125300,3.128915
350,3.109900,3.125694
400,3.099100,3.121910
450,3.083000,3.119586
500,3.071800,3.119801


/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarni

TrainOutput(global_step=1200, training_loss=3.0725308787822723, metrics={'train_runtime': 32968.9864, 'train_samples_per_second': 2.329, 'train_steps_per_second': 0.036, 'total_flos': 3.239066229276672e+17, 'train_loss': 3.0725308787822723})

# Let's test it out (Inference)

In [20]:
from peft import PeftModel
checkpoint = 'checkpoint-1200'
base_model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=device_map, torch_dtype="auto")
model = PeftModel.from_pretrained(base_model, checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/illjahunovan/poetry_env/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/home/illjahunovan/poetry_env/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [28]:
def generate_text(prompt: str, max_length: int = 40, temperature: float = 0.7, top_p: float = 0.9, seed: int | float | None = 52, **kwargs):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    set_seed(seed)
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            **kwargs
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_text_beam(prompt: str, 
                       max_length: int = 40, 
                       num_beams: int = 10, 
                       num_return_sequences: int = 3, 
                       no_repeat_ngram_size: int = 2, 
                       num_beam_groups: int = 2,
                       diversity_penalty: float = 1.0,
                       seed: int | float | None = 52,
                       **kwargs):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    set_seed(seed)
    model.eval()
    with torch.no_grad():
        sequences = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_beams=num_beams, # beam width 
            early_stopping=True, # stop when all beams finish
            no_repeat_ngram_size=no_repeat_ngram_size, # n-gram penalties, technique to avoid repetition
            num_return_sequences=num_return_sequences, # return top k sequences
            num_beam_groups=num_beam_groups, # divides num_beams to this number groups. two groups differ from each other. greatly combines with diversity_penalty 
            diversity_penalty=diversity_penalty, # encourages diversity beams (if using num_beam_groups)
            trust_remote_code=True, # beam search backward compatibility
            **kwargs
        )
    return tokenizer.batch_decode(sequences, skip_special_tokens=True)

In [29]:
outputs = generate_text_beam("Быть или не быть?", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=1, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.0,
                       seed=52)

In [30]:
print(outputs[0])

Быть или не быть? Вот вопрос, на который я не мог найти никакого ответа. Я не знал, что мне делать и как мне быть. Наконец я решился и пошел к Наташе. Она сидела одна и читала книгу. Увидев меня, она тотчас же встала со стула и подошла ко мне.  Что с вами? Вы нездоровы?  спросила она, смотря на меня с каким-то беспокойством и в то же время как будто с удивлением,  я думала, вы больны.


In [24]:
outputs = generate_text_beam("Бог ему судья!", 
                       max_length=100, 
                       num_beams=15, 
                       num_return_sequences=15, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=3,
                       diversity_penalty=1.0,
                       seed=52)
print(outputs[0], outputs[5], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Бог ему судья!  прибавил он, обращаясь к Алеше.  А ты, Алеша, не сердись на меня, что я так говорю,  продолжал он вдруг, как бы опомнившись, и с каким-то особенным выражением посмотрел на Алешу. Он был в чрезвычайном волнении и в то же время как будто и не в себе, но, однако ж, тотчас же взял себя в руки и продолжал как ни в чем не бывало:  Я, брат, в
______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [25]:
outputs = generate_text_beam("И все-таки, тварь ли я дрожащая или право имею?", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

И все-таки, тварь ли я дрожащая или право имею?  вскричал он вдруг в исступлении.  Право имею, право!  вскричала я, вскочив с места, и бросилась к нему на шею. Он крепко прижал меня к груди и поцеловал в лоб. Я была вне себя от радости, от счастья, что он любит меня, любит так же, как и я его. Но он тотчас же отстранил меня от себя и, смотря мне прямо в глаза, сказал:
_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [33]:
outputs = generate_text_beam("Какое лучшее определение человека?", 
                       max_length=130, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Какое лучшее определение человека?  спросил он, смотря на меня.  Я не знаю лучшего определения человека,  отвечал я с жаром и с каким-то особенным жаром, как будто я сам был виноват в том, что он так меня мучил, и, может быть, в то же самое время я был и виноват перед ним, потому что я не понимал его, не мог понять его и не хотел понять. Он слушал меня с напряженным вниманием, но в лице его не было ни малейшего признака какого-нибудь удивления. Я продолжал:  Человек, по-моему, есть существо, созданное для того, чтобы быть счастливым,
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [32]:
outputs = generate_text_beam("Сомневаюсь на правильном ли я пути...", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Сомневаюсь на правильном ли я пути...  пробормотал он, как бы в раздумье.  Нет, нет, не сомневайтесь!  вскричал я в восторге,  вы правы, вы совершенно правы! Вы не ошибаетесь, я не ошибся! Я именно так и думал, что вы не ошибетесь и что я именно такой человек, каким вы меня описали, и именно таким я и должен быть, потому что, если б я был другой, то я бы не мог быть таким
_________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [42]:
outputs = generate_text_beam("Русский весьма часто смеется там, где", 
                       max_length=28, 
                       num_beams=15, 
                       num_return_sequences=15, 
                       no_repeat_ngram_size=3, 
                       num_beam_groups=3,
                       diversity_penalty=1.0,
                       seed=52)
print(outputs[-1], sep='\n' + '_' * 1000 + '\n')

Русский весьма часто смеется там, где, казалось бы, должен бы плакать.  Да, это так,  подтвердил я.  Но,


In [43]:
outputs = generate_text_beam("Жизнь задыхается без", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Жизнь задыхается без вас!  вскричала она с отчаянием.  О, боже мой! Что с вами? Вы больны?  Не знаю,  отвечал он, смотря на нее с каким-то странным, как бы рассеянным видом. Он был бледен, губы его дрожали, и он не мог выговорить ни слова. Наконец он взял ее за руку и повел к себе в комнату. Она шла за ним, не зная, что с нею делается; она была в каком-
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [44]:
outputs = generate_text_beam("Главное, самому себе не лгите... ибо лгущий самому себе теряет способность", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Главное, самому себе не лгите... ибо лгущий самому себе теряет способность говорить правду.  Ну, это уж слишком!  вскричал князь,  вы, кажется, не знаете, что такое ложь?  Нет, знаю, и очень хорошо знаю. Ложь есть ложь, а правда есть правда. Если бы вы знали, как я ненавижу ложь и как люблю правду, то, может быть, вы бы мне и не поверили. Но вы не верите, потому что не хотите верить.
_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## Some conclusions after experimenting
- LoRA algorithm is amazing for fine-tuning large and medium size llms \
it provides a way to train even when other common memory saving techniques couldn't help out
- adafactor optimizer (though it wasn't used eventually) is a fantastic replacement for AdamW
- gradient_accumulation idea is a real saver for imitating large batch size
- quantinization and fp16 especially are musthave for nlp tasks
- iterable, streaming datasets combining with generators are light-weight and really handy   
- DataCollatorForLanguageModeling, return_overflowing_tokens, return_overflowing_tokens and other stuff from hf libraries rid of so much boilerplate code